In [1]:
import pandas as pd
import numpy as np

> Through a series processing, compressed 6 million rows of data into 20 thousands rows.

In [2]:
df = pd.read_csv('https://www.kaggle.com/ntnu-testimon/paysim1?select=PS_20174392719_1491204439457_log.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [3]:
df = df.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)
df.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0


In [4]:
df.shape

(6362620, 8)

In [7]:
#Storing the fraudulent data into a dataframe
df_fraud = df[df['isFraud'] == 1]

In [8]:
#Storing the non-fraudulent data into a dataframe 
df_nofraud = df[df['isFraud'] == 0]
#Storing 12,000 rows of non-fraudulent data
df_nofraud = df_nofraud.head(12000)
#Joining both datasets together 
df = pd.concat([df_fraud, df_nofraud], axis = 0)
df.shape

(20213, 8)

In [9]:
#Package Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#Converting the type column to categorical
df['type'] = df['type'].astype('category')

#Integer Encoding the 'type' column
type_encode = LabelEncoder()

#Integer encoding the 'type' column
df['type'] = type_encode.fit_transform(df.type)

In [10]:
df.head(120)

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
2,1,4,181.00,181.00,0.0,0.00,0.00,1
3,1,1,181.00,181.00,0.0,21182.00,0.00,1
251,1,4,2806.00,2806.00,0.0,0.00,0.00,1
252,1,1,2806.00,2806.00,0.0,26202.00,0.00,1
680,1,4,20128.00,20128.00,0.0,0.00,0.00,1
...,...,...,...,...,...,...,...,...
99372,10,1,503405.88,503405.88,0.0,29669.00,533074.88,1
102180,10,4,2662734.59,2662734.59,0.0,0.00,0.00,1
102181,10,1,2662734.59,2662734.59,0.0,14165.62,2930405.33,1
102607,10,4,9217.19,9217.19,0.0,0.00,96795.60,1


In [11]:
#One hot encoding the 'type' column
type_one_hot = OneHotEncoder()
type_one_hot_encode = type_one_hot.fit_transform(df.type.values.reshape(-1,1)).toarray()

#Adding the one hot encoded variables to the dataset 
ohe_variable = pd.DataFrame(type_one_hot_encode, columns = ["type_"+str(int(i)) for i in range(type_one_hot_encode.shape[1])])
df = pd.concat([df, ohe_variable], axis=1)

#Dropping the original type variable 
df = df.drop('type', axis = 1)

#Viewing the new dataframe after one-hot-encoding 
df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_0,type_1,type_2,type_3,type_4
0,1.0,9839.64,170136.0,160296.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,1864.28,21249.0,19384.72,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,181.00,181.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,181.00,181.0,0.00,21182.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1.0,11668.14,41554.0,29885.86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
df.isnull().any()

step              True
amount            True
oldbalanceOrg     True
newbalanceOrig    True
oldbalanceDest    True
newbalanceDest    True
isFraud           True
type_0            True
type_1            True
type_2            True
type_3            True
type_4            True
dtype: bool

In [13]:
df.fillna(0)

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_0,type_1,type_2,type_3,type_4
0,1.0,9839.64,170136.00,160296.36,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,1864.28,21249.00,19384.72,0.00,0.00,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,181.00,181.00,0.00,0.00,0.00,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,181.00,181.00,0.00,21182.00,0.00,1.0,0.0,1.0,0.0,0.0,0.0
4,1.0,11668.14,41554.00,29885.86,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743.0,339682.13,339682.13,0.00,0.00,339682.13,1.0,0.0,0.0,0.0,0.0,0.0
6362616,743.0,6311409.28,6311409.28,0.00,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0
6362617,743.0,6311409.28,6311409.28,0.00,68488.84,6379898.11,1.0,0.0,0.0,0.0,0.0,0.0
6362618,743.0,850002.52,850002.52,0.00,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df.to_csv('fraud_prediction.csv')